In [1]:
import os
import sys
import uuid

import random

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dataset preprocessing

In [2]:
vdjdb_raw = pd.read_csv('data/raw/vdjdb_paired_010923.tsv', sep='\t')
iedb_raw = pd.read_csv('data/raw/iedb_010923.csv')
mcpas_raw = pd.read_csv('data/raw/mcpas-tcr_010923.csv', encoding='latin1')

/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_36992/1072130901.py:2: DtypeWarning: Columns (8,10,11,15,16,17,18,20,21,22,23,25,30,31,36,37,44,45,46,47,48,49,50,51,52,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedb_raw = pd.read_csv('data/raw/iedb_010923.csv')
/var/folders/w1/mbcsphtx2xz1xqrls0v3d6r80000gn/T/ipykernel_36992/1072130901.py:3: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  mcpas_raw = pd.read_csv('data/raw/mcpas-tcr_010923.csv', encoding='latin1')


## VDJdb

In [3]:
vdj_cols = ['complex.id', 'CDR3', 'V', 'J', 'Species', 'MHC A','MHC class', 'Epitope']
print('VDJdb raw:', vdjdb_raw.shape)

vdjdb = vdjdb_raw.dropna()
vdjdb_a = vdjdb[vdjdb['Gene']=='TRA'][vdj_cols].copy()
vdjdb_b = vdjdb[vdjdb['Gene']=='TRB'][vdj_cols].copy()

vdjdb = pd.merge(vdjdb_a, vdjdb_b, on='complex.id')
print('VDJdb paired:', vdjdb.shape)

assert (vdjdb['Species_x'] == vdjdb['Species_y']).all()
assert (vdjdb['Epitope_x'] == vdjdb['Epitope_y']).all()
assert (vdjdb['MHC A_x'] == vdjdb['MHC A_y']).all()

vdj_cols = ['complex.id', 'CDR3_x', 'V_x', 'J_x', 'CDR3_y', 'V_y', 'J_y', 'Species_y', 'MHC A_y', 'MHC class_y', 'Epitope_y']
vdjdb = vdjdb[vdj_cols].copy()
vdj_cols = ['complex.id', 'cdr3a', 'va', 'ja', 'cdr3b', 'vb', 'jb', 'species', 'mhc', 'mhc_class', 'epitope']
vdjdb.columns = vdj_cols

# final data formating
try:
    vdjdb = vdjdb.drop('complex.id', axis=1)
except:
    pass
vdjdb = vdjdb.replace('MHCI', 1) # name formatting
vdjdb = vdjdb.replace('HomoSapiens', 'human') # name formatting
vdjdb = vdjdb[vdjdb['species']=='human'].copy() # keep only humans
vdjdb = vdjdb.drop_duplicates()
print("preprocessed vdjdb:", vdjdb.shape)
vdjdb.head()


VDJdb raw: (3050, 17)
VDJdb paired: (1397, 15)
preprocessed vdjdb: (1102, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
0,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEKGGL
1,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEKGGL
2,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,human,HLA-B*08,1,FLKEKGGL
3,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEQGGL
4,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,human,HLA-B*08,1,FLKEQGGL


## IEDB

IEDB queried for MHC 1 and human only

In [4]:
subset = ['Chain 1 CDR3 Curated', 'Chain 2 CDR3 Curated']
iedb = iedb_raw.dropna(subset=subset)
iedb = iedb.astype('str')
print("Full IEDB:", iedb.shape)

iedb_base_cols = ['Group Receptor ID', 'Receptor ID', 'Description', 'Organism', 'MHC Allele Names']

iedb_cols = [col for col in iedb.columns if "Curated" in col and ("CDR3" in col or "V" in col or "J" in col)]
iedb_cols = ['Description', 'Organism', 'MHC Allele Names', 'Curated Chain 1 V Gene', 'Curated Chain 1 J Gene', 'Chain 1 CDR3 Curated',  
            'Curated Chain 2 V Gene','Curated Chain 2 J Gene', 'Chain 2 CDR3 Curated']
iedb = iedb[iedb_cols]
print("IEDB Selected Columns:", iedb.shape)

iedb = iedb.replace('nan', np.nan)
iedb = iedb.dropna()
iedb.columns = ['epitope', 'species', 'mhc', 'va', 'ja', 'cdr3a', 'vb', 'jb', 'cdr3b']
iedb['mhc_class'] = 1
iedb['species'] = 'human' # IEDB filtered for only human

iedb = iedb[vdjdb.columns]
print('IEDB remove nan:', iedb.shape)
iedb = iedb.drop_duplicates()
print('IEDB remove duplicates:', iedb.shape)

# we need the CDR3 ends for 3D generation, one way to get this is to check if the sequence starts with C
iedb = iedb[iedb['cdr3a'].str.startswith('C')]
iedb = iedb[iedb['cdr3b'].str.startswith('C')]
print('IEDB with cdr3 ends only:', iedb.shape)

# VJ name formating for cases where V, J gene starts with 'TCR' instead of 'TR'
iedb = iedb.replace('TCR', 'TR', regex=True)
iedb

Full IEDB: (24032, 71)
IEDB Selected Columns: (24032, 9)
IEDB remove nan: (6093, 10)
IEDB remove duplicates: (6002, 10)
IEDB with cdr3 ends only: (5483, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
81,CAVRPTSGGSYIPTF,TRAV21*01,TRAJ6*01,CASSYVGNTGELFF,TRBV6-5*01,TRBJ2-2*01,human,HLA-A*02:01,1,SLLMWITQC
173,CAGGTGNQFYF,TRAV35*02,TRAJ49*01,CAISEVGVGQPQHF,TRBV10-3,TRBJ1-5*01,human,HLA-A*02:01,1,AAGIGILTV
617,CALSEAGTGGSYIPTF,TRAV19,TRAJ6,CASSMFVGQPQHF,TRBV19,TRBJ1-5,human,HLA-A*02:01,1,GILGFVFTL
618,CAVSVEETSGSRLTF,TRAV41,TRAJ58,CASSFFHNNEQFF,TRBV19,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL
619,CAYRSARDSSYKLIF,TRAV38-2/DV8*01,TRAJ12*01,CASSDHSVTGISSPLHF,TRBV7-9*03,TRBJ1-6*02,human,HLA-B7,1,TPRVTGGGAM
...,...,...,...,...,...,...,...,...,...,...
24299,CGAGETSGSRLTF,TRAV21,TRAJ58,CSVNLGGPTDTQYF,TRBV29-1,TRBJ2-3,human,HLA-B*07:02,1,KPVETSNSF
24300,CALEGSQGNLIF,TRAV9-2,TRAJ42,CSVPDGAEPYGYTF,TRBV20-1,TRBJ1-2,human,HLA-A*01:01,1,TTDPSFLGRY
24301,CLVGNTGGFKTIF,TRAV4,TRAJ9,CSVPDRGNTEAFF,TRBV29-1,TRBJ1-1,human,HLA-B*07:02,1,SPRWYFYYL
24303,CAPSRHAGNNRKLIW,TRAV9-2,TRAJ38,CSVQGGTNEKLFF,TRBV29-1,TRBJ1-4,human,HLA-A*01:01,1,VSDGGPNLY


## McPAS-TCR

In [5]:
mcpas_cols = ['CDR3.alpha.aa', 'CDR3.beta.aa', 'Species', 'Epitope.peptide', 'MHC', 'TRAV', 'TRAJ', 'TRBV', 'TRBJ']
mcpas = mcpas_raw[mcpas_cols].copy()
print('McPAS-TCR raw:', mcpas.shape)
mcpas = mcpas.dropna()
print('McPAS-TCR drop na:', mcpas.shape)
mcpas = mcpas.drop_duplicates()
print('McPAS-TCR drop na:', mcpas.shape)

# formatting
mcpas['Species'] = mcpas['Species'].str.lower()
mcpas.columns = ['cdr3a', 'cdr3b', 'species', 'epitope', 'mhc', 'va', 'ja', 'vb', 'jb']
mcpas['mhc_class'] = 1
mcpas = mcpas[vdjdb.columns].copy()
mcpas 

McPAS-TCR raw: (39985, 9)
McPAS-TCR drop na: (3124, 9)
McPAS-TCR drop na: (2914, 9)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope
73,CALGLMSNYNVLYF,TRAV4,TRAJ14-4,CASSSGLGGTLYF,TRBV10,TRBJ2-4,mouse,H-2Db,1,SSGVENPGGYCLTKW
80,CAAETTASLGKLQF,TRAV11,TRAJ9,CASGDHGLSYEQYF,TRBV13-3,TRBJ2-6,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
84,CALGDRGSGGSNYK,TRAV4,TRAJ3DT,CAWSRTGGNSDYTF,TRBV31,TRBJ1-2,mouse,H-2b,1,DEPLTSLTPRCNTAWNRLKL
120,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV8-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
121,CAAEASSSFSKLVF,TRAV11-1,TRAJ42,CASAPDRGGERLF,TRBV13-2,TRBJ1-4,mouse,H-2q,1,GPEGAQGPRGEPGTP
...,...,...,...,...,...,...,...,...,...,...
39028,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39029,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL
39030,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL
39031,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL


## Combine

In [6]:
positives = pd.concat((iedb, vdjdb, mcpas), axis=0)
print("concate db:", positives.shape)
positives = positives.drop_duplicates(subset=['cdr3a', 'cdr3b', 'epitope'], keep='last', ignore_index=True)
print("concate db remove duplicates:", positives.shape)
positives['uuid'] = [uuid.uuid4() for _ in range(len(positives.index))]

positives

concate db: (9499, 10)
concate db remove duplicates: (8433, 10)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope,uuid
0,CAVSVEETSGSRLTF,TRAV41,TRAJ58,CASSFFHNNEQFF,TRBV19,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL,918149ee-14d1-491e-8755-d6131a665fff
1,CAVNKGYGQNFVF,TRAV12-2*02,TRAJ26*01,CASSPAGISYNSPLHF,TRBV7-9*03,TRBJ1-6*01,human,HLA-B8,1,ELRRKMMYM,f5a0c861-7e57-4f9e-a71b-c636aed0a2f7
2,CAVRDSSYSGAGSYQLTF,TRAV3*01,TRAJ28*01,CASSRLAGASTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,1903ba15-3a56-4f54-8eff-5011e95955a4
3,CAVSDYGQNFVF,TRAV21*01,TRAJ26*01,CASSRLSSNTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,2e9bfee9-74d4-4673-b8e3-0a6abdfbb00e
4,CATAQVYSGGGADGLTF,TRAV17*01,TRAJ45*01,CASSRLAGNTDTQYF,TRBV7-3*01,TRBJ2-3*01,human,HLA-B8,1,QIKVRVDMV,b9b0e561-0c40-42a3-94d0-ca2bc7cc4909
...,...,...,...,...,...,...,...,...,...,...,...
8428,CALDGPSNTGKLIF,TRAV16,TRAJ37,CATSESSGQTYEQYF,TRBV15,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL,0c98bcf7-7275-46ef-af50-5db50966a970
8429,CATDAEGNNRLAF,TRAV17,TRAJ7,CASSIFGGGLGEQFF,TRBV19,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL,77497b56-72e9-411f-b6f6-acc07dfc82a3
8430,CGAVGYQKVTF,TRAV34,TRAJ13,CALNGEISYNEQFF,TRBV2,TRBJ2-2,human,HLA-A2:01,1,FLCMKALLL,747e84a3-0ed8-4ae9-9416-d816197da805
8431,CAVIWYNNNDMRF,TRAV8-1,TRAJ43,CASSQGVNTGELFF,TRBV4-2,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL,1c7f3671-fbd9-4bc2-adfd-9506061b2469


In [7]:
complex = []
for i in range(len(positives)):
    complex.append((positives['cdr3a'].iloc[i], positives['cdr3b'].iloc[i], positives['epitope'].iloc[i]))
positives['complex'] = complex
positives_dict = positives.drop_duplicates(subset='complex').set_index('complex', drop=True).to_dict(orient='index')
try:
    positives = positives.drop('complex', axis=1)
except:
    pass
print('unique tcr-peptide pairs:', len(positives_dict.keys()))

unique tcr-peptide pairs: 8433


# Negative sample generation

In [8]:
i = 0
negatives = []
while i<len(positives.index):
    sample_idx = random.choice([k for k in range(len(positives)) if k!=i])
    neg_complex = (positives['cdr3a'].iloc[sample_idx], positives['cdr3b'].iloc[sample_idx], positives['epitope'].iloc[i])
    try:
        # sample already exists and it's positive and we do not want this
        positives_dict[neg_complex]
    except:
        # negative sample is not positive
        sample = positives.iloc[sample_idx].copy()
        sample['epitope'] =  positives['epitope'].iloc[i]
        sample['mhc'] = positives['mhc'].iloc[i]
        if tuple(sample.values) not in negatives: # check if generated negative already exists
            negatives.append(tuple(sample.values))
            i+=1
negatives = pd.DataFrame(negatives, columns=positives.columns)
negatives['uuid'] = [uuid.uuid4() for _ in range(len(negatives.index))]
print('negative samples:', negatives.shape)
negatives

negative samples: (8433, 11)


,cdr3a,va,ja,cdr3b,vb,jb,species,mhc,mhc_class,epitope,uuid
0,CALSENTDKLIF,TRAV19,TRAJ34,CSARAELAADFISSYNEQFF,TRBV20-1,TRBJ2-1,human,HLA-A*02:01,1,GILGFVFTL,d3bddb1c-0b35-4187-8b44-5e1f946a90de
1,CAVRNNNNAPRF,TRAV7N-5:01,TRAJ4-01,CASSRDSAETLYF,TRBV1-01,TRBJ2-3:01,mouse,HLA-B8,1,ELRRKMMYM,cdc0a764-5d77-43b5-bcee-d1799c37d432
2,CVLSGTGYQNFYF,TRAV9N-2:01,TRAJ4-01,CASSMGTGFSDYTF,TRBV1-01,TRBJ1-2:01,mouse,HLA-B8,1,QIKVRVDMV,23332d11-4709-4ec2-a087-5df1e808598b
3,CVGGDDKIIF,TRAV12-1,TRAJ30,CASSQTLSYEQYF,TRBV4-1,TRBJ2-7,human,HLA-B8,1,QIKVRVDMV,ba1f399a-be76-45f8-9420-a4641e354b77
4,CVALQRYNFNKFYF,TRAV17,TRAJ21,CSARASGGHKQYEQYF,TRBV20-1,TRBJ2-7,human,HLA-B8,1,QIKVRVDMV,519c6d35-2038-4aa9-8ef2-4928c2ac04e0
...,...,...,...,...,...,...,...,...,...,...,...
8428,CATVQGAQKLVF,TRAV17,TRAJ54,CSASWNEQFF,TRBV29-1,TRBJ2-1,human,HLA-A2:01,1,FLCMKALLL,665cc567-e5af-4952-b8b0-352bafa5bdfe
8429,CAVSDDKIIF,TRAV12-2,TRAJ30,CASSDGTNSYEQYF,TRBV11-2,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL,d30db593-6dae-49a8-bc23-db23b76a7c44
8430,CAVNGDGQKLLF,TRAV12-2,TRAJ16,CAISEGGAYEQYF,TRBV10-3,TRBJ2-7,human,HLA-A2:01,1,FLCMKALLL,59b5d9ca-c3eb-40c7-aca3-038b0b460b4b
8431,CAVQGQNSGGYQKVTF,TRAV20*01,TRAJ13*01,CSATDRAASYNSPLHF,TRBV20-1*01,TRBJ1-6*01,human,HLA-A2:01,1,FLCMKALLL,972324d4-ed94-4e1e-ba9f-370e2dd32af9


# Saving Data

In [9]:
save_path = 'data/preprocessed'
positives.to_csv(os.path.join(save_path, 'positives.tsv'), sep='\t')
negatives.to_csv(os.path.join(save_path, 'negatives.tsv'), sep='\t')